In [1]:
# Project Libraries
import data

# Generic Libraries
import numpy as np
import pandas as pd

In [8]:
# ----- Useful dataframes
def dataframes():
    # Informative dataframe
    df_info = pd.DataFrame({
        'bid_volume': [0.0001],
        'ask_volume': [0.0001],
        'commission': [0.000025]
    })

    # Oders dataframe
    df_orders = pd.DataFrame(columns = ['timestamp', 'order_id', 'side', 'price', 'order_amount', 'symbol'])

    # Trades dataframe
    df_trades = pd.DataFrame(columns = ['timestamp', 'order_id', 'trade_id', 'side', 'price', 'symbol', 'filled_amount', 'order_amount', 'fee'])

    # Inventory dataframe
    df_inventory = pd.DataFrame({
        'timestamp': [0],
        'base (BTC)': [5],
        'quote (USDT)': [500000]
    })

    # Rebalance dataframe
    df_rebalances = pd.DataFrame(columns = ['timestamp', 'sent', 'received', 'price', 'fee'])

    return df_info, df_orders, df_trades, df_inventory, df_rebalances


def names(name):
    if name == 'bid':
        return 'buy'
    elif name == 'ask':
        return 'sell'
    else:
        'Function Failure: names'

# Position valuation
def filled_volume(size_variation: float, price_variation: float, operation: str = None, constant_position_base: float = .0001):
    # Ask
    if operation == 'ask':
        if price_variation > 0:
            current_position = constant_position_base
        elif price_variation == 0:
            if size_variation > 0:
                current_position = 0
            else:
                if size_variation <= constant_position_base:
                    current_position = size_variation
                else:
                    current_position = constant_position_base
        else:
            current_position = 0
    # Bid
    if operation == 'bid':
        if price_variation < 0:
            current_position = constant_position_base
        elif price_variation == 0:
            if size_variation > 0:
                current_position = 0
            else:
                if size_variation <= constant_position_base:
                    current_position = size_variation
                else:
                    current_position = constant_position_base
        else:
            current_position = 0
    return current_position

def execute_trade(side,i, current_id, traded_volume, trade_fee, first_input = None):
    df_trades.at[i,'timestamp'] = df_bitfinex_tob.iloc[i]['timestamp']
    df_trades.at[i,'order_id'] = df_orders[df_orders['side'] == names(side)].iloc[-1]['order_id']
    df_trades.at[i,'trade_id'] = current_id
    df_trades.at[i,'side'] = side
    df_trades.at[i,'price'] = df_orders[df_orders['side'] == names(side)].iloc[-1]['price']
    df_trades.at[i,'symbol'] = 'BTC/USDT'
    df_trades.at[i,'filled_amount'] = traded_volume
    df_trades.at[i,'order_amount'] = df_orders[df_orders['side'] == names(side)].iloc[-1]['order_amount']
    df_trades.at[i,'fee'] = trade_fee * traded_volume * df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['price']

    if first_input == True:
        df_output = pd.DataFrame([
        df_bitfinex_tob.iloc[1]['timestamp'],
        df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['order_id'],
        1, side_filled, df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['price'], 'BTC/USDT',
        filled, df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['order_amount'],
        platform_fee * filled * df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['price']
    ], index = ['timestamp','order_id','trade_id','side','price','symbol','filled_amount','order_amount','fee']
    ).transpose()
        return df_output

def place_order(i, side, current_id, amount):
    df_orders.at[i, 'timestamp'] = df_bitfinex_tob.iloc[i]['timestamp']
    df_orders.at[i, 'order_id'] = current_id
    df_orders.at[i, 'side'] = names(side)
    df_orders.at[i, 'price'] = df_bitfinex_tob.iloc[i][side]
    df_orders.at[i, 'order_amount'] = amount
    df_orders.at[i, 'symbol'] = 'BTC/USDT'

In [9]:
# ----- Market information
df_bitfinex_tob = pd.DataFrame(data.read_file(file_name = "orderbooks_05jul21.json", folder_route = "files/")['bitfinex']).transpose().reset_index()
df_bitfinex_tob['index'] = pd.to_datetime(df_bitfinex_tob['index'])
df_bitfinex_tob = df_bitfinex_tob.set_index('index').resample('min').last().ffill()
for column in df_bitfinex_tob.columns:
    df_bitfinex_tob[column] = df_bitfinex_tob[column].apply(lambda x: x['0'])
df_bitfinex_tob = df_bitfinex_tob.reset_index().rename(columns = {'index':'timestamp'})

# SLA Conditions
contracted_volume = .0001
platform_fee = .000025

# Initial Order
df_orders = pd.DataFrame([
    [df_bitfinex_tob.iloc[0]['timestamp'], df_bitfinex_tob.iloc[0]['timestamp']],
    [1,2], ['buy','sell'], [df_bitfinex_tob.iloc[0]['bid'], df_bitfinex_tob.iloc[0]['ask']],
    [contracted_volume, contracted_volume], ['BTC/USDT','BTC/USDT']
    ], 
    index =  ['timestamp','order_id','side','price', 'order_amount','symbol'],
    columns = [0,1]
).transpose()
df_orders

,timestamp,order_id,side,price,order_amount,symbol
0,2021-07-05 13:06:00+00:00,1,buy,28282.0,0.0001,BTC/USDT
1,2021-07-05 13:06:00+00:00,2,sell,28283.0,0.0001,BTC/USDT


In [10]:
for side_filled in ['ask','bid']:
    filled = filled_volume(
        size_variation = df_bitfinex_tob.iloc[1][side_filled+'_size'] - df_bitfinex_tob.iloc[0][side_filled+'_size'],
        operation = side_filled, 
        price_variation = df_bitfinex_tob.iloc[1][side_filled] - df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['price'], 
        constant_position_base = contracted_volume
    )
    print(filled)
    if filled > 0:
            df_trades = execute_trade(
                side = side_filled,
                i = 1, 
                current_id = 1, 
                traded_volume = filled, 
                trade_fee = platform_fee, 
                first_input = True)
            
            # ---- Liquidity Pool Adjustment

            # ----

            place_order(
                i = 1,
                side = side_filled,
                current_id = df_orders.iloc[-1]['order_id'] + 1, 
                amount = contracted_volume
                )
            
            # ---- Liquidity Pool Adjustment

            # ----
df_orders


0.0001


NameError: name 'df_trades' is not defined

In [19]:
df_trades = execute_trade(
                df_orders = df_orders,
                side = 'buy',
                i = 1, 
                current_id = 2, 
                traded_volume = .0001, 
                trade_fee = .000025, 
                first_input = True)
df_trades

,timestamp,order_id,trade_id,side,price,symbol,filled_amount,order_amount,fee
0,2021-07-05 13:07:00+00:00,2,1,ask,28283.0,BTC/USDT,0.0001,0.0001,0.000071


In [17]:
def execute_trade(df_orders,side,i, current_id, traded_volume, trade_fee, first_input = True):
    if first_input == True:
        df_output = pd.DataFrame([
        df_bitfinex_tob.iloc[1]['timestamp'],
        df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['order_id'],
        1, side_filled, df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['price'], 'BTC/USDT',
        filled, df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['order_amount'],
        platform_fee * filled * df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['price']
    ], index = ['timestamp','order_id','trade_id','side','price','symbol','filled_amount','order_amount','fee']
    ).transpose()
        return df_output
    elif first_input == False:
        df_trades.at[i,'timestamp'] = df_bitfinex_tob.iloc[i]['timestamp']
        df_trades.at[i,'order_id'] = df_orders[df_orders['side'] == names(side)].iloc[-1]['order_id']
        df_trades.at[i,'trade_id'] = current_id
        df_trades.at[i,'side'] = side
        df_trades.at[i,'price'] = df_orders[df_orders['side'] == names(side)].iloc[-1]['price']
        df_trades.at[i,'symbol'] = 'BTC/USDT'
        df_trades.at[i,'filled_amount'] = traded_volume
        df_trades.at[i,'order_amount'] = df_orders[df_orders['side'] == names(side)].iloc[-1]['order_amount']
        df_trades.at[i,'fee'] = trade_fee * traded_volume * df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['price']

        

In [26]:
def execute_trade(df_orders,side,i, current_id, traded_volume, trade_fee, first_input = True):
    if first_input == True:
        df_output = pd.DataFrame([
        df_bitfinex_tob.iloc[1]['timestamp'],
        df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['order_id'],
        1, side_filled, df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['price'], 'BTC/USDT',
        filled, df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['order_amount'],
        platform_fee * filled * df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['price']
    ], index = ['timestamp','order_id','trade_id','side','price','symbol','filled_amount','order_amount','fee']
    ).transpose()
        return df_output
    elif first_input == False:
        df_trades.at[i,'timestamp'] = df_bitfinex_tob.iloc[i]['timestamp']
        df_trades.at[i,'order_id'] = df_orders[df_orders['side'] == names(side)].iloc[-1]['order_id']
        df_trades.at[i,'trade_id'] = current_id
        df_trades.at[i,'side'] = side
        df_trades.at[i,'price'] = df_orders[df_orders['side'] == names(side)].iloc[-1]['price']
        df_trades.at[i,'symbol'] = 'BTC/USDT'
        df_trades.at[i,'filled_amount'] = traded_volume
        df_trades.at[i,'order_amount'] = df_orders[df_orders['side'] == names(side)].iloc[-1]['order_amount']
        df_trades.at[i,'fee'] = trade_fee * traded_volume * df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['price']

        

In [16]:
# ----- Market information
df_bitfinex_tob = pd.DataFrame(data.read_file(file_name = "orderbooks_05jul21.json", folder_route = "files/")['bitfinex']).transpose().reset_index()
df_bitfinex_tob['index'] = pd.to_datetime(df_bitfinex_tob['index'])
df_bitfinex_tob = df_bitfinex_tob.set_index('index').resample('min').last().ffill()
for column in df_bitfinex_tob.columns:
    df_bitfinex_tob[column] = df_bitfinex_tob[column].apply(lambda x: x['0'])
df_bitfinex_tob = df_bitfinex_tob.reset_index().rename(columns = {'index':'timestamp'})

In [77]:
# ----- Market information
df_bitfinex_tob = pd.DataFrame(data.read_file(file_name = "orderbooks_05jul21.json", folder_route = "files/")['bitfinex']).transpose().reset_index()
df_bitfinex_tob['index'] = pd.to_datetime(df_bitfinex_tob['index'])
df_bitfinex_tob = df_bitfinex_tob.set_index('index').resample('min').last().ffill()
for column in df_bitfinex_tob.columns:
    df_bitfinex_tob[column] = df_bitfinex_tob[column].apply(lambda x: x['0'])
df_bitfinex_tob = df_bitfinex_tob.reset_index().rename(columns = {'index':'timestamp'})

# SLA Conditions
contracted_volume = .0001
platform_fee = .000025

# Initial Order
df_orders = pd.DataFrame([
    [df_bitfinex_tob.iloc[0]['timestamp'], df_bitfinex_tob.iloc[0]['timestamp']],
    [1,2], ['buy','sell'], [df_bitfinex_tob.iloc[0]['bid'], df_bitfinex_tob.iloc[0]['ask']],
    [contracted_volume, contracted_volume], ['BTC/USDT','BTC/USDT']
    ], 
    index =  ['timestamp','order_id','side','price', 'order_amount','symbol'],
    columns = [0,1]
).transpose()

for side_filled in ['ask','bid']:
    filled =filled_volume(
        size_variation = df_bitfinex_tob.iloc[1][side_filled+'_size'] - df_bitfinex_tob.iloc[0][side_filled+'_size'],
        operation = side_filled, 
        price_variation = df_bitfinex_tob.iloc[1][side_filled] - df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['price'], 
        constant_position_base = contracted_volume
    )

    if filled > 0:
            df_trades = execute_trade(
                side = side_filled,
                i = 1, 
                current_id = 1, 
                traded_volume = filled, 
                trade_fee = platform_fee, 
                first_input = True)
            
            # ---- Liquidity Pool Adjustment

            # ----

            place_order(
                i = 1,
                side = side_filled,
                current_id = df_orders.iloc[-1]['order_id'] + 1, 
                amount = contracted_volume
                )
            
            # ---- Liquidity Pool Adjustment

            # ----



AttributeError: 'NoneType' object has no attribute 'at'

In [53]:
# Operation
period = 2

for side_filled in ['ask','bid']:
    filled =filled_volume(
        size_variation = df_bitfinex_tob.iloc[1][side_filled+'_size'] - df_bitfinex_tob.iloc[0][side_filled+'_size'],
        operation = side_filled, 
        price_variation = df_bitfinex_tob.iloc[1][side_filled] - df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['price'], 
        constant_position_base = contracted_volume
    )

    if filled > 0:
            last_id = df_trades['trade_id'].iloc[-1]
            df_trades = execute_trade(
                side = side_filled,
                i = period,
                current_id = last_id + 1, 
                traded_volume = filled, 
                trade_fee = platform_fee
                )
            
            # ---- Liquidity Pool Adjustment

            # ----

            place_order(
                i = period,
                side = side_filled,
                current_id = df_orders.iloc[-1]['order_id'] + 1, 
                amount = contracted_volume
                )
            
            # ---- Liquidity Pool Adjustment

            # ----


df_trades

2

In [44]:
df_bitfinex_tob

,timestamp,ask_size,ask,bid,bid_size
0,2021-07-05 13:06:00+00:00,0.100600,28283.0,28282.0,0.338172
1,2021-07-05 13:07:00+00:00,2.792849,28325.0,28323.0,0.594797
2,2021-07-05 13:08:00+00:00,0.403360,28315.0,28309.0,0.093481
3,2021-07-05 13:09:00+00:00,0.148733,28333.0,28329.0,0.339650
4,2021-07-05 13:10:00+00:00,0.003524,28341.0,28336.0,3.807731
...,...,...,...,...,...
56,2021-07-05 14:02:00+00:00,3.550600,28292.0,28288.0,0.446627
57,2021-07-05 14:03:00+00:00,0.385010,28301.0,28295.0,0.600127
58,2021-07-05 14:04:00+00:00,0.009931,28352.0,28340.0,0.357682
59,2021-07-05 14:05:00+00:00,0.445649,28361.0,28355.0,0.006000


In [24]:
execute_trade(
    side = side_filled,
    i = 1,
    current_id = df_trades.iloc[-1]['trade_id'],
    traded_volume = filled,
    trade_fee = platform_fee
)

In [39]:
def execute_trade(side,i, current_id, traded_volume, trade_fee, first_input = None):
    df_trades.at[i,'timestamp'] = df_bitfinex_tob.iloc[i]['timestamp']
    df_trades.at[i,'order_id'] = df_orders[df_orders['side'] == names(side)].iloc[-1]['order_id']
    df_trades.at[i,'trade_id'] = current_id
    df_trades.at[i,'side'] = side
    df_trades.at[i,'price'] = df_orders[df_orders['side'] == names(side)].iloc[-1]['price']
    df_trades.at[i,'symbol'] = 'BTC/USDT'
    df_trades.at[i,'filled_amount'] = traded_volume
    df_trades.at[i,'order_amount'] = df_orders[df_orders['side'] == names(side)].iloc[-1]['order_amount']
    df_trades.at[i,'fee'] = trade_fee * traded_volume * df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['price']

    if first_input == True:
        df_output = pd.DataFrame([
        df_bitfinex_tob.iloc[1]['timestamp'],
        df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['order_id'],
        1, side_filled, df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['price'], 'BTC/USDT',
        filled, df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['order_amount'],
        platform_fee * filled * df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['price']
    ], index = ['timestamp','order_id','trade_id','side','price','symbol','filled_amount','order_amount','fee']
    ).transpose()
        return df_output

In [36]:
df_bitfinex_tob

,timestamp,ask_size,ask,bid,bid_size
0,2021-07-05 13:06:00+00:00,0.100600,28283.0,28282.0,0.338172
1,2021-07-05 13:07:00+00:00,2.792849,28325.0,28323.0,0.594797
2,2021-07-05 13:08:00+00:00,0.403360,28315.0,28309.0,0.093481
3,2021-07-05 13:09:00+00:00,0.148733,28333.0,28329.0,0.339650
4,2021-07-05 13:10:00+00:00,0.003524,28341.0,28336.0,3.807731
...,...,...,...,...,...
56,2021-07-05 14:02:00+00:00,3.550600,28292.0,28288.0,0.446627
57,2021-07-05 14:03:00+00:00,0.385010,28301.0,28295.0,0.600127
58,2021-07-05 14:04:00+00:00,0.009931,28352.0,28340.0,0.357682
59,2021-07-05 14:05:00+00:00,0.445649,28361.0,28355.0,0.006000


In [ ]:
filled_volume(size_variation = , price_variation: float, operation: str = None, constant_position_base: float = .0001)

In [12]:
pd.DataFrame(data.read_file(file_name = "orderbooks_05jul21.json", folder_route = "files/")['bitfinex']).transpose().reset_index().iloc[0]['ask']

{'0': 28275.0,
 '1': 28276.0,
 '2': 28277.0,
 '3': 28278.0,
 '4': 28280.0,
 '5': 28281.0,
 '6': 28282.0,
 '7': 28283.0,
 '8': 28284.0,
 '9': 28285.0,
 '10': 28286.0,
 '11': 28287.0,
 '12': 28288.0,
 '13': 28289.0,
 '14': 28290.0,
 '15': 28291.0,
 '16': 28292.0,
 '17': 28293.0,
 '18': 28294.0,
 '19': 28295.0,
 '20': 28296.0,
 '21': 28297.0,
 '22': 28298.0,
 '23': 28299.0,
 '24': 28300.0}